## Deliverable 2. Create a Customer Travel Destinations Map.

In [41]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Date
0,0,Charagua,-19.8000,-63.2167,89.17,48,58,8.23,broken clouds,2022-02-02 22:30:46
1,1,Ushuaia,-54.8000,-68.3000,44.26,57,75,25.32,broken clouds,2022-02-02 22:30:46
2,2,Mataura,-46.1927,168.8643,58.48,97,100,3.96,moderate rain,2022-02-02 22:28:57
3,3,Winslow,35.0242,-110.6974,41.18,23,100,5.01,overcast clouds,2022-02-02 22:30:47
4,4,Lebu,-37.6167,-73.6500,63.72,79,45,11.01,scattered clouds,2022-02-02 22:29:34


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the lowest daily high temperture you would like for your vacation?"))
max_temp = float(input("What is the highest daily high temperture you would like for your vacation?"))

What is the lowest daily high temperture you would like for your vacation?75
What is the highest daily high temperture you would like for your vacation?76


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
trip_df=city_data_df.loc[(city_data_df["Max Temp"]>= min_temp) & (city_data_df["Max Temp"]<= max_temp)]
trip_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Date
6,6,Nabire,-3.3667,135.4833,75.58,89,100,2.73,overcast clouds,2022-02-02 22:25:54
18,18,Santa Ines,-3.6667,-45.3800,75.78,89,98,5.19,overcast clouds,2022-02-02 22:30:50
30,30,Kalmunai,7.4167,81.8167,75.18,73,83,3.83,broken clouds,2022-02-02 22:30:52
32,32,Dingle,10.9995,122.6711,75.81,90,100,9.13,overcast clouds,2022-02-02 22:30:53
90,90,Komatipoort,-25.4332,31.9548,75.06,92,0,1.32,clear sky,2022-02-02 22:33:09


In [5]:
# 4a. Determine if there are any empty rows.
trip_df.isnull().sum()


City_ID        0
City           0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
Date           0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
trip_df = trip_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = trip_df[["City", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Max Temp,Description,Lat,Lng,Hotel Name
6,Nabire,75.58,overcast clouds,-3.3667,135.4833,
18,Santa Ines,75.78,overcast clouds,-3.6667,-45.3800,
30,Kalmunai,75.18,broken clouds,7.4167,81.8167,
32,Dingle,75.81,overcast clouds,10.9995,122.6711,
90,Komatipoort,75.06,clear sky,-25.4332,31.9548,
118,Cidreira,75.67,clear sky,-30.1811,-50.2056,
140,Meulaboh,75.81,broken clouds,4.1363,96.1285,
152,Inhambane,75.88,few clouds,-23.8650,35.3833,
289,Valparaiso,75.43,clear sky,-33.0393,-71.6273,
291,Maues,75.34,overcast clouds,-3.3836,-57.7186,


In [8]:
for index,row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    locations = [lat,lng]
    print(locations)


[-3.3667, 135.4833]
[-3.6667, -45.38]
[7.4167, 81.8167]
[10.9995, 122.6711]
[-25.4332, 31.9548]
[-30.1811, -50.2056]
[4.1363, 96.1285]
[-23.865, 35.3833]
[-33.0393, -71.6273]
[-3.3836, -57.7186]
[-43.249, -65.3051]
[-16.2325, 39.9086]
[-18.5067, -54.76]
[-7.1911, -48.2072]
[-8.1842, -76.5125]
[-15.4311, -74.6158]
[-28.8667, 153.5667]
[14.8961, -24.4956]
[-2.4959, -66.2662]
[-10.6517, -68.5044]
[-5.3166, 119.3661]


In [21]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}

# 6b. Iterate through the hotel DataFrame.
for index,row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    locations = [lat,lng]
    params["location"]=f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    print(f"Retrieving Results for Index {index}.")
    hotels = requests.get(base_url, params=params).json()
    results = hotels['results']
     # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels['results'][0]['name']
    except (IndexError):
        print("Hotel not found... Skipping.")
        

Retrieving Results for Index 6.
Retrieving Results for Index 18.
Retrieving Results for Index 30.
Retrieving Results for Index 32.
Retrieving Results for Index 90.
Retrieving Results for Index 118.
Retrieving Results for Index 140.
Retrieving Results for Index 152.
Retrieving Results for Index 289.
Retrieving Results for Index 291.
Retrieving Results for Index 398.
Retrieving Results for Index 401.
Retrieving Results for Index 415.
Retrieving Results for Index 430.
Retrieving Results for Index 452.
Retrieving Results for Index 468.
Retrieving Results for Index 493.
Retrieving Results for Index 511.
Retrieving Results for Index 554.
Hotel not found... Skipping.
Retrieving Results for Index 680.
Retrieving Results for Index 691.


In [26]:
hotel_df


,City,Max Temp,Description,Lat,Lng,Hotel Name
6,Nabire,75.58,overcast clouds,-3.3667,135.4833,Hotel Nusantara Nabire
18,Santa Ines,75.78,overcast clouds,-3.6667,-45.3800,Palace hotel
30,Kalmunai,75.18,broken clouds,7.4167,81.8167,VS Villa
32,Dingle,75.81,overcast clouds,10.9995,122.6711,SEGAYA PROPERTY
90,Komatipoort,75.06,clear sky,-25.4332,31.9548,Kruger View Backpackers
118,Cidreira,75.67,clear sky,-30.1811,-50.2056,Hotel Castelo
140,Meulaboh,75.81,broken clouds,4.1363,96.1285,Kost Pria
152,Inhambane,75.88,few clouds,-23.8650,35.3833,Oceano
289,Valparaiso,75.43,clear sky,-33.0393,-71.6273,Hotel Ultramar
291,Maues,75.34,overcast clouds,-3.3836,-57.7186,Hotel Miramar


In [55]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.replace('', np.nan)
hotel_df =hotel_df.dropna()

In [13]:
# 8a. Create the output File (CSV)

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

NameError: name 'clean_hotel_df' is not defined

In [ ]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [ ]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
